In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import subprocess

> **Note:** This cell below is optional.  
> The bash script often printed extra comments, which cluttered the output.  
> Running this cell simply helps keep the output cleaner.

In [2]:
import os

# Strip out the exported bash functions that break Jupyter subshells
for key in list(os.environ):
    if key.startswith("BASH_FUNC_"):
        del os.environ[key]

print("✅ Cleaned exported bash functions. Subshells will be quiet now.")


✅ Cleaned exported bash functions. Subshells will be quiet now.


> Sometimes you may want to run code in a different directory than where this `.ipynb` notebook is located.  
> The cell below sets the working directory for everything you run after executing it.

In [3]:
# set working directory for all future cells
os.chdir("/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run


'/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run'

> **Important:**  
> These scripts are tailored for **ACCRE** and load modules available on ACCRE at Vanderbilt University.
> 
> **Option 1 – Adjust for your HPC:**  
> Edit the `module load` lines to match the Amber modules you would like to use
> **Option 2 – Source Amber.sh:**  
> If your cluster doesn’t use modules, remove those lines and source the AMBER environment , e.g.:
> ```bash
> source /sb/apps/amber22/amber.sh
> ```
> This ensures the correct AMBER version is used on any cluster.


In [4]:
import os, subprocess

def load_amber_env():
    """Load AmberTools + Amber environment into the Jupyter kernel."""
    bash_cmd = """
    module purge
    module load StdEnvACCRE/2023
    module load gcc/12.3 openmpi/4.1.5 cuda/12.2
    module load ambertools/25.0
    module load scipy-stack/2025a
    module load amber/24
    env
    """
    env_output = subprocess.check_output(["bash", "-c", bash_cmd], text=True)
    for line in env_output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v
    print("✅ Amber environment loaded into Jupyter kernel.")


> Run the next cell to load the **AmberTools + Amber environment** into your Jupyter kernel.  
> This ensures all subsequent commands use the correct modules and environment variables.


In [5]:
load_amber_env()

The following modules were not unloaded:
  (Use "module --force purge" to unload all):

  1) CCconfig        6)  ucx/1.14.1         11) flexiblas/3.3.1
  2) gentoo/2023     7)  libfabric/1.18.0   12) imkl/2023.2.0
  3) gcccore/.12.3   8)  pmix/4.2.4         13) StdEnvACCRE/2023
  4) gcc/12.3        9)  ucc/1.2.0
  5) hwloc/2.9.1     10) openmpi/4.1.5
Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2023b
(required by: plumed/2.9.0, ambertools/25.0)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) ipykernel/2023b => ipykernel/2025a
  2) scipy-stack/2023b => scipy-stack/2025a



✅ Amber environment loaded into Jupyter kernel.


Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2025a
(required by: amber/24)
-------------------------------------------------------------------------------




Inactive Modules:
  1) arrayfire/3.9.0     2) cudnn/8.9.5.29     3) glfw/3.3.8     4) spdlog/1.11.0

Due to MODULEPATH changes, the following have been reloaded:
  1) ambertools/25.0     2) plumed/2.9.0     3) ucc-cuda/1.2.0     4) ucx-cuda/1.14.1

The following have been reloaded with a version change:
  1) cuda/12.2 => cuda/12.6
  2) cudacore/.12.2.2 => cudacore/.12.6.2
  3) ipykernel/2025a => ipykernel/2023b
  4) nccl/2.18.3 => nccl/2.26.2
  5) scipy-stack/2025a => scipy-stack/2023b



### Prepare the Non-Covalent Ligand

Rosiglitazone (BRL) from the 6ONJ PDB is **not protonated**.  

1. Create a new directory called `ligand_prep` and change into it.  
2. Copy `BRL.pdb` (BRL saved as a PDB file without PPARg LBD in 6ONJ.pdb) into this directory for further processing.


In [8]:
%cd ligand_prep
!reduce BRL.pdb > BRL_H.pdb

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run/ligand_prep
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'


/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Processing file: "BRL.pdb"
Building or keeping OH & SH Hydrogens.
VDW dot density = 16/A^2
Probe radius = 0.25A
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Rotating NH3 Hydrogens.
Not processing Met methyls.
Found 0 hydrogens (0 hets)
Standardized 0 hydrogens (0 hets)
Added 19 hydrogens (19 hets)
If you publish work which uses reduce, please cite:
Word, et. al. (1999) J. Mol. Biol. 285, 1735-1747.
For more information see http://kinemage.biochem.duke.edu


Use `pdb4amber` to clean and prepare `BRL_H.pdb` for AMBER compatibility, saving the output as `BRL_clean_H.pdb`.


In [9]:
!pdb4amber -i BRL_H.pdb -o BRL_clean_H.pdb
#Upload onto RED server and generate the charges 

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'

Summary of pdb4amber for: BRL_H.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames
BRL

---------- Missing heavy atom(s)

None


Upload the files to the **R.E.D. Server**, run the job, then download the entire job folder and place it in your working directory.

***********************

Generate the `BRL.frcmod` file (force field parameters) from the `.mol2` file using GAFF2:


In [11]:
!parmchk2 -i ../RED_server_P7173/Data-R.E.D.Server/Mol_m1/Mol-sm_m1-c1.mol2 -f mol2 -o BRL.frcmod -s gaff2

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'


Create the `tleap.in` input file (notice that we are cd-ing to the parent directory).  
This script instructs **tleap** to load the ligand and protein, apply the chosen force fields, add counter-ions, and build the final system topology and coordinate files for AMBER.


In [12]:
%%bash
# Write tleap.in file for building solvated system

cd ../

cat > tleap.in <<'EOF'
#!/bin/bash
# load script using: tleap -s -f tleap.in

source leaprc.protein.ff14SB
source leaprc.gaff
source leaprc.water.tip3p

loadamberparams ligand_prep/BRL.frcmod
BRL = loadmol3 RED_server_P7173/Data-R.E.D.Server/Mol_m1/Mol-sm_m1-c1.mol2 
complex = loadPdb 6ONJ_BRL_complex.pdb

check complex
charge complex

addions complex Na+ 0
addions complex Cl- 0

solvateoct complex TIP3PBOX 10.0 iso

# addions complex K+ 11
# addions complex Cl- 11

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb complex prepin.pdb

quit

EOF

echo "✅ tleap.in file written successfully."


bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


✅ tleap.in file written successfully.


In [ ]:
#Change the notebook’s working directory to the parent folder and run `tleap`.
%cd ../
!tleap -s -f tleap.in

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/prep to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm to search path.
-I: Adding /cvmfs/soft.computeca

/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Loading parameters: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib/amino12.lib
Loading library: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib/aminoct12.lib
Loading library: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib/aminont12.lib
----- Source: /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/amber

Display the last line of `prepin.inpcrd` to find the simulation box dimensions generated by `tleap`.

In [16]:
!tail -n 1 prepin.inpcrd

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
  81.1891231  81.1891231  81.1891231 109.4712190 109.4712190 109.4712190


Use `molarity.perl` (from the `Scripts` folder) to calculate the amount of KCl to add for a **50 mM solution**  
based on the given box dimensions:


In [18]:
#perl molarity.perl {conc in M} {pasted coordinates from above}
!perl Scripts/molarity.perl 0.05 81.1891231  81.1891231  81.1891231 109.4712190 109.4712190 109.4712190

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
 MOLARITY =    0.050
 Box size =   81.189   81.189   81.189  109.471  109.471  109.471
 Volume = 411975.783

   12.405 molecules are necessary to make a molarity of   0.05 M



The molarity calculation output indicated that ~12.4 molecules are needed for a 0.05 M solution so we will add 12 K+ and 12 Cl- to the tleap.in file

Update the `tleap.in` file to adjust the number of ions added (K⁺ and Cl⁻) based on the molarity calculation:


In [23]:
# Replace K+ 4 -> K+ 11 and Cl- 11 -> Cl- 16 in tleap input file
!sed -i 's/# addions complex K+ 11/addions complex K+ 12/; s/# addions complex Cl- 11/addions complex Cl- 12/' tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'


Sanity check - confirm that the intended changes were made 

In [24]:
!tail 10 tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
tail: cannot open '10' for reading: No such file or directory
==> tleap.in <==
solvateoct complex TIP3PBOX 10.0 iso

addions complex K+ 12
addions complex Cl- 12

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb complex prepin.pdb

quit



Re-run tleap so we have input files with all the updates 

In [25]:
!tleap -s -f tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/prep to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat

### Run Minimization & Equilibration

This creates a `Minimization` directory, moves into it, and runs `mesmy.sh`.  

The `mesmy.sh` script performs a full **system relaxation protocol** using `pmemd.cuda`:

- **Step 1–5: Energy Minimization**  
  - Strongly restrains heavy atoms, then gradually relaxes the restraints until unrestrained.  
  - Uses steepest descent minimization to relieve steric clashes and relax solvent.

- **Step 6–9: Equilibration**  
  - Runs short NVT (constant volume) and NPT (constant pressure) MD simulations.  
  - Gradually reduces restraints on the protein and backbone atoms.  
  - Slowly increases timestep and stabilizes density and temperature.  

Each step writes:
- Restart files (`.rst7`) – used as input for the next step  
- Trajectories (`.nc`) – for visualization and analysis  
- Output logs (`.out`, `mdinfo`) – for monitoring progress  

By the end, the system is minimized, equilibrated, and ready for production MD.


In [26]:
!mkdir Minimization 
%cd Minimization 

!sh ../Scripts/mesmy.sh > log.mesmy

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run/Minimization
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following flo

### Hydrogen Mass Repartitioning (HMR)

This cell creates a small ParmEd input file (`hmr_commands.in`) that performs **Hydrogen Mass Repartitioning**  
and writes out a new topology file `prepinHMR.prmtop`.  
HMR redistributes a small amount of mass from heavy atoms to hydrogens,  
reducing calculations and making the simulation faster

In [27]:
%%bash
# Create a ParmEd input file with HMR commands
cat > hmr_commands.in <<EOF
HMassRepartition
outparm prepinHMR.prmtop
go
quit
EOF

# Run ParmEd with the input file
parmed ../prepin.prmtop < hmr_commands.in

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'



                                  /^^^/           /]
                                 /   ]           / ]
                         _______/    ]___       /  ]
                        /                \_____/   /
                      _/   [@]  \ \                \
                     /..         | |                ]
                      VVVvvv\    | |         _/\    ]
          P A R M E D       |               /    \  ]
                     AAA^^^^^              /       \]
                      \_________\   \_____/
                                \    \
                                 \____\

ParmEd: a Parameter file Editor


Loaded Amber topology file ../prepin.prmtop
Reading input from STDIN...
> Repartitioning hydrogen masses to 3.024 daltons. Not changing water hydrogen masses.
> Outputting Amber topology file prepinHMR.prmtop
> Done!


Create 'Production' folder and copy required files from Scripts folder

In [ ]:
%cd ../
!mkdir Production
%cd Production
!cp ../Scripts/production.pl .
!cp ../Scripts/prod1.in.start .
!cp ../Scripts/prod1.in .

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_noncovalent_run/Production
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'


/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/ipykernel/2025a/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'


### Production MD Setup & Execution

Production MD is handled by a **Perl driver script** (`production.pl`) that automates running multiple  
**100 ns blocks** of MD back-to-back using `pmemd.cuda`. This allows you to build very long trajectories  
(e.g., microseconds) without manually restarting each run.

What `production.pl` does:
- Reads the final minimized/equilibrated coordinates (`step9.rst7`) and topology (`prepinHMR.prmtop`).
- Loops from `$nstart` to `$nend`, generating sequential 100 ns MD chunks:
  - Uses `prod1.in.start` for the first run (fresh start) and `prod1.in` for restarts.
  - Automatically names output files (`PTR_001.out`, `md_prod1_001.nc`, `md_prod1_001.rst`, etc.).
  - Chains restart files so each chunk continues from where the previous one ended.
- Prints and executes the `pmemd.cuda` command for each run.

Example:
- `$nend = 10` → runs 10 × 100 ns = **1 μs total production MD**.
- Increase `$nend` to extend the trajectory (e.g., 20 → 2 μs total).

The input files (`prod1.in.start` and `prod1.in`) specify:
- **Explicit solvent MD** under NPT conditions  
- **dt = 0.004 ps** (with SHAKE and HMR applied → 4 fs timestep)  
- Langevin thermostat (`ntt = 3`) and Monte Carlo barostat (`barostat = 2`)  
- Coordinates written every 10 ps (`ntwx = 2500`)  

> **Tip:**  
> Edit `production.pl` to adjust the number of chunks, file paths, or output naming as needed  
> before starting long production runs.


In [ ]:
#Test to see if it works 
!perl production.pl > PRODUCTION.OUT

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
^C


### Submit Production MD to the GPU Cluster

This cell writes a **SLURM job submission script** (`6ONI_submit_slurm`) in the current working directory.  
It requests a GPU node on ACCRE, loads the necessary modules, and runs `production.pl` to launch  
the automated multi-chunk production MD.

- **For ACCRE users:** The script is ready to submit with `sbatch 6ONI_submit_slurm`.  
- **For other HPC systems:**  
  - Update the `#SBATCH` directives (partition, account, GPU request) to match your scheduler.  
  - If your cluster does not use modules, remove the `module load` commands and instead  
    source the Amber environment manually, e.g.:  
    ```bash
    source /path/to/amber.sh
    ```


In [ ]:
%%bash
# Write slrum script in the working directory, later submit to ACCRE 

cat > 6ONI_submit_slurm <<'EOF'
#!/bin/bash
#SBATCH --job-name=Amber_Apo_6ONI_Run
#SBATCH --mail-type=ALL
#SBATCH --nodes=1
#SBATCH --mem=40G
#SBATCH --time=5-00:00:00
#SBATCH --partition=batch_gpu
#SBATCH --account=csb_gpu_acc
#SBATCH --gres=gpu:nvidia_titan_x:1   # request exactly 3 GPUs on this node
#SBATCH --output=PPARg_meta_out_%j.log
#SBATCH --error=PPARg_meta_err_%j.log

module purge
module load StdEnvACCRE/2023
module load gcc/12.3 openmpi/4.1.5 cuda/12.2
module load ambertools/25.0
module load scipy-stack/2025a
module load amber/24
    
perl production.pl > PRODUCTION.OUT

EOF

In [ ]:
!sbatch 6ONI_submit_slurm